In [1]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('train_users_2.csv.zip', compression='zip')
sessions_df = pd.read_csv('sessions.csv.zip', compression='zip')

In [3]:
sessions_df['id'] = sessions_df['user_id']
sessions_df = sessions_df.drop(['user_id'],axis=1)
sessions_df

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NaN,NaN,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NaN,NaN,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NaN,NaN,Windows Desktop,435.0,d1mm9tcy42
...,...,...,...,...,...,...
10567732,dashboard,view,dashboard,Windows Desktop,556.0,9uqfg8txu3
10567733,edit,view,edit_profile,Windows Desktop,6624.0,9uqfg8txu3
10567734,webcam_upload,-unknown-,-unknown-,Windows Desktop,200125.0,9uqfg8txu3
10567735,active,-unknown-,-unknown-,-unknown-,17624.0,9uqfg8txu3


In [4]:
len(sessions_df['id'].unique())

135484

In [5]:
test = sessions_df[sessions_df['action_detail'] == 'post_checkout_action']
len(test['id'].unique())

7521

In [6]:
sessions_data = sessions_df[['action_detail', 'device_type', 'secs_elapsed', 'id']]

In [7]:
sessions_data['secs_elapsed'].fillna(0, inplace=True)
sessions_data['action_detail'].fillna('Not explicitly stated', inplace=True)

E:\Anaconda3\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [8]:
sessions_list = ['action_detail', 'device_type']

In [9]:
def del_rares(df, column_list, threshold):
    n = threshold * len(df['id'].unique())
    for c in column_list:
        df = df[df[c].isin(df[c].value_counts()[df[c].value_counts() > n].index)]
    return df

In [10]:
sessions_data = del_rares(sessions_data, sessions_list, 0.01)

In [11]:
#new_data = sessions_data.copy()

In [12]:
categorical_list = ['message_to_host_focus', 'view_listing',
       'update_listing_description', 'listing_descriptions',
       'message_to_host_change', 'manage_listing', 'create_listing',
       'pending', 'p5', 'post_checkout_action']

In [13]:
def apply_categorical(df, column_list):
    for c in column_list:
        df[c] = np.where(df['action_detail']==c,1,0)
    return df

In [14]:
apply_categorical(sessions_data, categorical_list)
sessions_data

,action_detail,device_type,secs_elapsed,id,message_to_host_focus,view_listing,update_listing_description,listing_descriptions,message_to_host_change,manage_listing,create_listing,pending,p5,post_checkout_action
0,Not explicitly stated,Windows Desktop,319.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0
1,view_search_results,Windows Desktop,67753.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0
2,Not explicitly stated,Windows Desktop,301.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0
3,view_search_results,Windows Desktop,22141.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0
4,Not explicitly stated,Windows Desktop,435.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10567732,dashboard,Windows Desktop,556.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0
10567733,edit_profile,Windows Desktop,6624.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0
10567734,-unknown-,Windows Desktop,200125.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0
10567735,-unknown-,-unknown-,17624.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0


In [15]:
sessions_data['hours_elapsed'] = sessions_data['secs_elapsed']/3600
sessions_data

,action_detail,device_type,secs_elapsed,id,message_to_host_focus,view_listing,update_listing_description,listing_descriptions,message_to_host_change,manage_listing,create_listing,pending,p5,post_checkout_action,hours_elapsed
0,Not explicitly stated,Windows Desktop,319.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0,0.088611
1,view_search_results,Windows Desktop,67753.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0,18.820278
2,Not explicitly stated,Windows Desktop,301.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0,0.083611
3,view_search_results,Windows Desktop,22141.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0,6.150278
4,Not explicitly stated,Windows Desktop,435.0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,0,0.120833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10567732,dashboard,Windows Desktop,556.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0,0.154444
10567733,edit_profile,Windows Desktop,6624.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0,1.840000
10567734,-unknown-,Windows Desktop,200125.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0,55.590278
10567735,-unknown-,-unknown-,17624.0,9uqfg8txu3,0,0,0,0,0,0,0,0,0,0,4.895556


In [16]:
new_sessions_data = pd.DataFrame(list(sessions_data['id'].unique()), columns=['id'])
new_sessions_data

,id
0,d1mm9tcy42
1,yo8nz8bqcq
2,4grx6yxeby
3,ncf87guaf0
4,4rvqpxoh3h
...,...
135457,cv0na2lf5a
135458,zp8xfonng8
135459,fa6260ziny
135460,87k0fy4ugm


In [17]:
c1 = sessions_data.groupby('id')['device_type'].max().to_frame().reset_index()
c2 = sessions_data.groupby('id')['hours_elapsed'].agg(sum).to_frame().reset_index()
c3 = sessions_data.groupby('id')['message_to_host_focus'].max().to_frame().reset_index()
c4 = sessions_data.groupby('id')['view_listing'].max().to_frame().reset_index()
c5 = sessions_data.groupby('id')['update_listing_description'].max().to_frame().reset_index()
c6 = sessions_data.groupby('id')['listing_descriptions'].max().to_frame().reset_index()
c7 = sessions_data.groupby('id')['message_to_host_change'].max().to_frame().reset_index()
c8 = sessions_data.groupby('id')['manage_listing'].max().to_frame().reset_index()
c9 = sessions_data.groupby('id')['create_listing'].max().to_frame().reset_index()
c10 = sessions_data.groupby('id')['pending'].max().to_frame().reset_index()
c11 = sessions_data.groupby('id')['p5'].max().to_frame().reset_index()
c12 = sessions_data.groupby('id')['post_checkout_action'].max().to_frame().reset_index()

In [18]:
new_sessions_data = pd.merge(c1,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c2,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c3,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c4,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c5,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c6,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c7,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c8,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c9,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c10,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c11,new_sessions_data,how='inner',on='id')
new_sessions_data = pd.merge(c12,new_sessions_data,how='inner',on='id')

In [19]:
new_sessions_data.columns

Index(['id', 'post_checkout_action', 'p5', 'pending', 'create_listing',
       'manage_listing', 'message_to_host_change', 'listing_descriptions',
       'update_listing_description', 'view_listing', 'message_to_host_focus',
       'hours_elapsed', 'device_type'],
      dtype='object')

In [20]:
train_df.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

In [21]:
train_df['date_account_created'] = pd.to_datetime(train_df['date_account_created'], format='%Y-%m-%d')

In [22]:
len(train_df)

213451

In [23]:
def extract_dates(df):
    Y,M,D,W=[],[],[],[]
    for i in range(len(df['date_account_created'])):
        the_day=df['date_account_created'][i]
        year=the_day.year
        month=the_day.month
        day=the_day.day
        weekday=the_day.dayofweek
        
        Y.append(year)
        M.append(month)
        D.append(day)
        W.append(weekday)

    df['year_account_created']=Y
    df['month_account_created']=M
    df['day_account_created']=D
    df['weekday_account_created']=W
    return df

In [24]:
extract_dates(train_df)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,year_account_created,month_account_created,day_account_created,weekday_account_created
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,2010,6,28,0
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,2011,5,25,2
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,2010,9,28,1
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,2011,12,5,0
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,2010,9,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,NaN,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF,2014,6,30,0
213447,mhewnxesx9,2014-06-30,20140630235719,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF,2014,6,30,0
213448,6o3arsjbb4,2014-06-30,20140630235754,NaN,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF,2014,6,30,0
213449,jh95kwisub,2014-06-30,20140630235822,NaN,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF,2014,6,30,0


In [25]:
train_df.loc[train_df['age'] > 1000, 'age'] = 2014 - train_df['age']

In [26]:
train_df.loc[(train_df['age'] < 18) | (train_df['age'] > 110), 'age'] = np.nan

In [27]:
train_df['age'].fillna(train_df['age'].mean(), inplace=True)

In [28]:
train_df['first_affiliate_tracked'].fillna('Not explicitly stated', inplace=True)

In [29]:
train_df.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                             0
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked         0
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
year_account_created            0
month_account_created           0
day_account_created             0
weekday_account_created         0
dtype: int64

In [30]:
train_data = train_df.drop(['timestamp_first_active', 'date_first_booking', 'date_account_created','signup_flow'], axis=1)

In [31]:
train_data.columns

Index(['id', 'gender', 'age', 'signup_method', 'language', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser', 'country_destination',
       'year_account_created', 'month_account_created', 'day_account_created',
       'weekday_account_created'],
      dtype='object')

In [32]:
train_list = ['gender', 'signup_method', 'language', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser', 'country_destination',
       'year_account_created', 'month_account_created', 'day_account_created',
       'weekday_account_created']

In [33]:
def del_rares(df, column_list, threshold):
    n = threshold * len(df['id'].unique())
    for c in column_list:
        df = df[df[c].isin(df[c].value_counts()[df[c].value_counts() > n].index)]
    return df

In [34]:
train_data = del_rares(train_data, train_list, 0.01)

In [35]:
train_data

,id,gender,age,signup_method,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,year_account_created,month_account_created,day_account_created,weekday_account_created
0,gxn3p5htnn,-unknown-,37.440629,facebook,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,2010,6,28,0
1,820tgsjxq7,MALE,38.000000,facebook,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,2011,5,25,2
2,4ft3gnwmtx,FEMALE,56.000000,basic,en,direct,direct,untracked,Web,Windows Desktop,IE,US,2010,9,28,1
3,bjjt8pjhuk,FEMALE,42.000000,facebook,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,2011,12,5,0
4,87mebub9p4,-unknown-,41.000000,basic,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,2010,9,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,MALE,32.000000,basic,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF,2014,6,30,0
213447,mhewnxesx9,-unknown-,37.440629,basic,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF,2014,6,30,0
213448,6o3arsjbb4,-unknown-,32.000000,basic,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF,2014,6,30,0
213449,jh95kwisub,-unknown-,37.440629,basic,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF,2014,6,30,0


In [36]:
train_data['language'].value_counts()

en    186787
Name: language, dtype: int64

In [37]:
train_data = train_data.drop(['language'], axis=1)

In [38]:
train_data = pd.merge(train_data,new_sessions_data,how='inner',on='id')

In [39]:
train_data.columns

Index(['id', 'gender', 'age', 'signup_method', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser', 'country_destination',
       'year_account_created', 'month_account_created', 'day_account_created',
       'weekday_account_created', 'post_checkout_action', 'p5', 'pending',
       'create_listing', 'manage_listing', 'message_to_host_change',
       'listing_descriptions', 'update_listing_description', 'view_listing',
       'message_to_host_focus', 'hours_elapsed', 'device_type'],
      dtype='object')

In [40]:
categorical_features = [
    'gender', 'signup_method', 'affiliate_channel',
     'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
     'first_device_type', 'first_browser', 'year_account_created', 'device_type',
     'month_account_created', 'day_account_created', 'weekday_account_created' 
]
train_data = pd.get_dummies(train_data, columns=categorical_features)

In [41]:
train_data 

,id,age,country_destination,post_checkout_action,p5,pending,create_listing,manage_listing,message_to_host_change,listing_descriptions,...,day_account_created_29,day_account_created_30,day_account_created_31,weekday_account_created_0,weekday_account_created_1,weekday_account_created_2,weekday_account_created_3,weekday_account_created_4,weekday_account_created_5,weekday_account_created_6
0,d1mm9tcy42,62.000000,other,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,yo8nz8bqcq,37.440629,NDF,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,4grx6yxeby,37.440629,NDF,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,ncf87guaf0,37.440629,NDF,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4rvqpxoh3h,37.440629,GB,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64128,zxodksqpep,32.000000,NDF,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
64129,mhewnxesx9,37.440629,NDF,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
64130,6o3arsjbb4,32.000000,NDF,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
64131,jh95kwisub,37.440629,NDF,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [42]:
train_data['country_destination'] = np.where(train_data['country_destination']!='NDF',1,0)

In [43]:
train_data

,id,age,country_destination,post_checkout_action,p5,pending,create_listing,manage_listing,message_to_host_change,listing_descriptions,...,day_account_created_29,day_account_created_30,day_account_created_31,weekday_account_created_0,weekday_account_created_1,weekday_account_created_2,weekday_account_created_3,weekday_account_created_4,weekday_account_created_5,weekday_account_created_6
0,d1mm9tcy42,62.000000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,yo8nz8bqcq,37.440629,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,4grx6yxeby,37.440629,0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,ncf87guaf0,37.440629,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4rvqpxoh3h,37.440629,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64128,zxodksqpep,32.000000,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
64129,mhewnxesx9,37.440629,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
64130,6o3arsjbb4,32.000000,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
64131,jh95kwisub,37.440629,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [44]:
train_data.set_index('id', inplace=True)

In [45]:
train_data.to_csv('../train_data.csv')